In [1]:
import pickle
from tqdm import tqdm

from neo4j import GraphDatabase
from neo4_connection import USER, PWD, URL
from utils import preprocess_neo4j_db

In [2]:
driver = GraphDatabase.driver(uri=URL, auth=(USER, PWD))

In [3]:
preprocess_neo4j_db(file_path='./neo4j_cypher_queries/01_db_preprocessing.txt')

In [4]:
with open('data/movie_recommendations_20230108', 'rb') as pkl:
    movie_recs = pickle.load(pkl)

type(movie_recs), len(movie_recs)

(list, 671)

In [6]:
movie_recs

[{'user': 0,
  'user_id_neo4j': 1,
  'rec_movie_ids': [8276,
   7527,
   8312,
   8971,
   5717,
   8839,
   8932,
   8756,
   8933,
   5427,
   6912,
   7793,
   8805,
   6725,
   7258,
   8862,
   8376,
   5727,
   8868,
   8510],
  'rec_movie_ids_neo4j': [100553,
   77658,
   102217,
   140265,
   26003,
   129514,
   136445,
   118890,
   136447,
   8484,
   58425,
   86377,
   127098,
   53883,
   69529,
   130970,
   104069,
   26082,
   131724,
   108709],
  'pred_rating': [3.8584861755371094,
   3.636410713195801,
   3.5989441871643066,
   3.5911216735839844,
   3.572518825531006,
   3.571625232696533,
   3.567986488342285,
   3.5546069145202637,
   3.537824869155884,
   3.515583038330078,
   3.475325584411621,
   3.468614101409912,
   3.455812931060791,
   3.455404281616211,
   3.4374475479125977,
   3.437101125717163,
   3.432352066040039,
   3.4310922622680664,
   3.427546977996826,
   3.4247374534606934]},
 {'user': 1,
  'user_id_neo4j': 2,
  'rec_movie_ids': [8276,
   7527

In [7]:
def create_recommendedTo_relationships(movie_recommendations:list) -> None:
    """ Creates relationships :RECOMMENDED_TO between movies and users.
        movie_recommendations (list): List of dicts with keys:
            user (int)
            user_id_neo4j (int)
            rec_movie_ids (list)
            rec_movie_ids_neo4j (list)
            pred_rating (list). 
    """
    with driver.session() as session:
        for rec in tqdm(movie_recs):

            user_id = rec['user_id_neo4j']
            rec_movie_ids = rec['rec_movie_ids_neo4j']
            query = f"""
            MATCH (u:User) 
            MATCH (m:Movie)
            WHERE u.userId = {user_id}
            AND m.movieId IN {rec_movie_ids}
            MERGE (m) -[:RECOMMENDED_TO]-> (u)
            ;
            """

            result = session.run(query)


create_recommendedTo_relationships(movie_recs)

100%|██████████| 671/671 [01:39<00:00,  6.74it/s]
